<a href="https://colab.research.google.com/github/SalvatoreRomano1/net2020/blob/main/Post_Group/Topic_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt

from scipy import sparse

In [2]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Data load and subframe division

In [11]:
# import the csv file with post and display the pandas dataframe
'''post = pd.read_csv(r"/content/gdrive/My Drive/com_liwc.csv", sep='\t', engine='python')
dp = pd.DataFrame(data=post)

pd.set_option('display.max_columns', None)
dp'''

# import the csv file with comment and display the pandas dataframe
comm = pd.read_csv(r"/content/gdrive/My Drive/com_liwc.csv", sep='\t', engine='python')
df = pd.DataFrame(data=comm)

pd.set_option('display.max_columns', None)
df

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,C'è poco da dire questa è gente che la guerra ...,1,FRATELLIDITALIA.PUGLIA,Se i musulmani pensano di portare la guerra sa...,problematico,probl-hate,incivile,negativo,Rifugiati Musulmani,NaN,Other,Other,4,2.0,50.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.00,0.0,0.0,125.00,100.00,0.00,0.0,0.0,0.00,25.00,0.0,0.0,0.00,0.0,0.00,6,6.00,33.33,33.33,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,33.33,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.0,0.0,0.0,16.67,16.67,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non p

In [12]:
#dataframe extraction by political side
'''dx_sx = df['p_dx_sx'].unique().tolist()
print(dx_sx)
df_0 = df[df['p_dx_sx'] == 'destra']
df_1 = df[df['p_dx_sx'] == 'sinistra']'''
#dataframe extraction by campaign
campaign = df['p_camapagna2'].unique().tolist()
print(campaign)
df_0 = df[df['p_camapagna2'] == 'Neg-comp']
df_1 = df[df['p_camapagna2'] == 'Positiva']

['Neg-comp', 'Positiva']


In [ ]:
#dataframe extraction based on the found values
'''rating = df['c_rating'].unique().tolist()
print(rating)
df_0= df[df['c_rating'] == 'problematico']
df_1 = df[df['c_rating'] == 'positivo']
df = df[df['c_rating'] == 'negativo']
df_3 = df[df['c_rating'] == 'hate']
df_4 = df[df['c_rating'] == 'ambiguo']'''

In [ ]:
#remove other topic and reset subdataframe index
df_0 = df_0[~df_0.c_topic.str.contains("Other")]
df_0 = df_0.reset_index()
df_0 = df_0.drop('index',axis = 1)
df_0

df_1 = df_1[~df_1.c_topic.str.contains("Other")]
df_1 = df_1.reset_index()
df_1 = df_1.drop('index',axis = 1)
df_1

Useful functions

In [15]:
#topic extraction
def feature(df):
  feature = df['c_topic'].unique().tolist() #use if the argument in the cell present more than one feaure than you want to keep separate
  feature1 = set()
  for i in range(len(feature)):
    temp = feature[i]
    temp = temp.split(' ')
    for j in range(len(temp)):
        feature1.add(temp[j])
  feature1 = list(feature1)
  return feature1

#biadjacency graph creation
def biadjacency(df,feature):
  size_df= len(df)
  matrix = np.zeros((len(feature),size_df)) 

  for i in range(size_df):
    feature_search = df['c_topic'][i]  #change if changing the column of interest
    feature_search= list(feature_search.split(" ")) 
    for j in range(len(feature_search)):
      temp = feature_search[j]
      index = feature.index(temp)
      matrix[index][i]+=1 

  matrix_sparse = sparse.csr_matrix(matrix)
  biadjacency_graph = nx.bipartite.from_biadjacency_matrix(matrix_sparse)

  return biadjacency_graph

#projection with respect to topic #feature or df?
def proj(feature,biad_graph):
  nodes = []
  for i in range(len(feature)):
    nodes.append(i)
  projection = nx.bipartite.projection.weighted_projected_graph(biad_graph, nodes)

  return projection

#bipartite nodes and edges creation
def bipartite_att(df,feature,biad_graph):
  set1 = pd.DataFrame(feature,columns =['Label'])
  set1['type']="topic"  
  #l = [x for x in range(len(feature1))]
  #set1['Id'] = l

  set2 = df[['c_topic',"p_PARTITO", 'p_politician','c_Emo_Neg','c_Rabbia']]  #change or add attribute attribute 
  set2.reset_index(inplace=True)
  set2['type'] = 'comment'  
  set2.rename(columns={ 'p_PARTITO': 'partito', 'p_politician': 'politician','c_Emo_Neg':'Emo_Neg','c_Rabbia':'Rabbia'}, inplace=True) #change attribute names

  nodes = set1.append(set2)

  l = [x for x in range(len(df)+len(feature))]
  nodes['Id'] = l

  edge = nx.to_pandas_edgelist(biad_graph)

  return nodes,edge

#projection nodes and edges creation
def proj_att(feature,projection):
  nodes = pd.DataFrame({'Label': feature})
  l = [x for x in range(len(nodes))]
  nodes['Id'] = l
  nodes['topic'] = feature

  edge = nx.to_pandas_edgelist(projection)

  
  return nodes,edge


Graphs

In [ ]:
#bipartite graph
topic = feature(df_0)
biad_graph = biadjacency(df_0,topic)
nodes, edges = bipartite_att(df_0,topic,biad_graph)

nodes.to_csv("nodelist_bipartite.csv",index=False)
edges.to_csv("edgelist_bipartite.csv",index=False)

In [19]:
#projection graph
topic = feature(df_0)
biad_graph = biadjacency(df_0,topic)
proj_graph = proj(topic,biad_graph)
nodes, edges = proj_att(topic,proj_graph)

nodes.to_csv("nodelist_projection.csv",index=False)
edges.to_csv("edgelist_projection.csv",index=False)